# Creating static web-sites using AWS S3 Object Storage

You should first have
- installed the awscli package to provide the aws command and also
- configured either
  - the ~/.aws/configure file with your AWS account credentials or
  - created a sourceable ~/.aws/credentials.rc (can be in any location) file

```
> cat ~/.aws/credentials.rc

export AWS_ACCESS_KEY_ID="<your-access-key>"
export AWS_SECRET_ACCESS_KEY="<your-secret-access-key>"
export AWS_DEFAULT_REGION=us-west-1
```

If you have chosen to use an rc file, source it as ```source <your-aws-credentials-rc-file>```, e.g.

In [70]:
. ~/.aws/credentials.rc

We can now use the aws cli utility to access S3 commands.

Let's investigate the available commands with ```aws s3 help```

In [71]:
aws s3 help 

S3()                                                                      S3()



NAME
       s3 -

DESCRIPTION
       This  section  explains  prominent concepts and notations in the set of
       high-level S3 commands provided.

   Path Argument Type
       Whenever using a command, at least one path argument must be specified.
       There are two types of path arguments: LocalPath and S3Uri.

       LocalPath: represents the path of a local file or directory.  It can be
       written as an absolute path or relative path.

       S3Uri: represents the location of a S3 object, prefix, or bucket.  This
       must  be  written in the form s3://mybucket/mykey where mybucket is the
       specified S3 bucket, mykey is the specified S3 key.  The path  argument
       must  begin with s3:// in order to denote that the path argument refers
       to a S3 object. Note that prefixes are separated  by  forward  slashes.
       For  example, if the S3 object myobject had the prefix myprefix,

S3 is organized into buckets, each of which contains a hierarchy of named objects.

The bucket name itself must be globally **unique** - across all AWS accounts not just yours.

Let's see if we have any buckets of our own using the ```aws s3 ls``` command.
You won't have any buckets if you just created your account.

In [72]:
aws s3 ls

Let's create our own bucket, we use the 'mb' or *make bucket* command to do this as
    ```aws s3 mb s3://<my-bucket-name>```
    
Note that we always address buckets in URL form ```s3://<bucket>```

In [73]:
aws s3 mb s3://mybucket

make_bucket failed: s3://mybucket An error occurred (BucketAlreadyExists) when calling the CreateBucket operation: The requested bucket name is not available. The bucket namespace is shared by all users of the system. Please select a different name and try again.


: 1

Of course someone got there first with our "mybucket" name !

So create a unique name, e.g. for myself:

In [74]:
aws s3 mb s3://mjbright-static-site

make_bucket: mjbright-static-site


In [75]:
aws s3 ls

2019-01-24 06:41:23 mjbright-static-site


That was lucky !

Now let's add some files to our bucket.

We can do this with the cp or sync commands.

Let's create an HTML index file and copy this into our bucket.

In [76]:
mkdir -p website;

cat > website/index.html <<EOF
<html>
<body>
    <h1> My first amazing web site !! </h1>
</body>
</html>
EOF

ls -al website/index.html

-rw-rw-r-- 1 user1 user1 74 Jan 24 06:41 website/index.html


In [77]:
aws s3 sync ./website s3://mjbright-static-site

upload: website/index.html to s3://mjbright-static-site/index.html


In [78]:
aws s3 ls s3://mjbright-static-site

2019-01-24 06:41:41         74 index.html


So it seems we have created a static web site.

We can use the handy S3 command website to declare that this is a website.

In [79]:
aws s3 website s3://mjbright-static-site --index-document index.html

The site should be available at http://<bucketname>.s3-website-<region>.amazonaws.com, which in this case would be:
    
    http://mjbright-static-site.s3-website-us-west-1.amazonaws.com/
    
However, if we visit that web page we will get an error telling us that we cannot access the page

In [80]:
wget -O - http://mjbright-static-site.s3-website-us-west-1.amazonaws.com/index.html

--2019-01-24 06:41:59--  http://mjbright-static-site.s3-website-us-west-1.amazonaws.com/index.html
Resolving mjbright-static-site.s3-website-us-west-1.amazonaws.com (mjbright-static-site.s3-website-us-west-1.amazonaws.com)... 52.219.20.66
Connecting to mjbright-static-site.s3-website-us-west-1.amazonaws.com (mjbright-static-site.s3-website-us-west-1.amazonaws.com)|52.219.20.66|:80... connected.
HTTP request sent, awaiting response... 403 Forbidden
2019-01-24 06:41:59 ERROR 403: Forbidden.



: 8

In fact we first need to enable website hosting from the bucket **and** enable public access to the index.html file.

We can do this via the AWS Console.

Connect to the console with your credentials and then navigate to https://s3.console.aws.amazon.com/s3/buckets/.


![](images/BucketProperties.JPG)

You should see your bucket listed here.

Click on the line (not on the bucket name which is a link, but under the Access or Region column) to see the following dropdown menu

![](images/BucketProperties-BeforeWebHostingEnabled.JPG)

Click on "Enable Web hosting" and you should see:

<!-- ![](images/BucketProperties-Settings-EnableWebsiteHosting.JPG) -->

![](images/BucketProperties-Settings-Enabled_WebsiteHosting.JPG)

But we still cannot access our site as we need to enable public access to the index.html file.

Click on the bucket name to be taken to a list of files in the bucket, select the index.html file and then "*Make Public*" in the dropdown "*Actions*" menu:

![](images/Make_index_public.JPG)



You now should be able to access your site using a browser, or from the command-line:

In [ ]:
wget -q -O - http://mjbright-static-site.s3-website-us-west-1.amazonaws.com/index.html

Let's now create something a little more like a website.

We'll use the Pelican command (installed as a Python module).


In [ ]:
pelican -o PELICAN -q

cp -a PELICAN/* website/

aws s3 sync website/ s3://mjbright-static-site/

In [ ]:
wget -q -O - http://mjbright-static-site.s3-website-us-west-1.amazonaws.com/index.html

Unfortunately this broke our website as we have not yet set permissions for the new files we just added to our website.

Go back to the bucket list at  https://s3.console.aws.amazon.com/s3/buckets/, drill down into your website bucket.

Then select all items within the bucket and from the Access drop-down menu select "*Make Public*"

Your website should now be accessible

In [ ]:
wget -q -O - http://mjbright-static-site.s3-website-us-west-1.amazonaws.com/index.html

# More Reading

You can find more details about S3 website hosting here: https://docs.aws.amazon.com/AmazonS3/latest/dev/WebsiteHosting.html

An article describing the use of S3 for static website hosting including use of https, DNS routing
https://medium.freecodecamp.org/how-to-host-a-static-website-with-s3-cloudfront-and-route53-7cbb11d4aeea

# Cleanup

Note that we can use the ```aws s3 rm``` command to remove files from the bucket and ```aws s3 rb``` command to remove a bucket.

In [ ]:
aws s3 rm s3://mjbright-static-site --recursive
aws s3 rb s3://mjbright-static-site

It's also possible to remoce the bucket directly using the ```--force``` option:
    ```aws s3 rb --force s3://mjbright-static-site```